In [1]:

import requests
import datetime
import pandas as pd
import time

max_tries = 3

url = 'https://apidatos.ree.es/es/datos/demanda/evolucion'

anios = range(2011,2026)
registros = []
lista_de_geo_ids = [8744,8745,4,5,6,7,8,9,10,11,13,14,15,16,17,8743,8742,20,21]

for anio in anios:
    for geo_id in lista_de_geo_ids:

        start_date = f"{anio}-01-01T00:00"
        end_date   = f"{anio}-12-31T23:59"

        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": "day",
            "geo_limit": "ccaa",
            "geo_ids": geo_id
        }

        for intento in range(max_tries):
            try:
                print(f"Pidiendo {anio}, geo_id {geo_id}...")
                resp = requests.get(url, params=params,timeout=10)
                resp.raise_for_status()
                data = resp.json()
                for v in data["included"][0]["attributes"]["values"]:
                    registros.append({
                        "datetime": v["datetime"],
                        "value": v["value"],
                        "geo_id": geo_id
                })
                break
            except (requests.RequestException,ValueError) as e:
                print(f"Error en {anio},geo_id {geo_id}: {e}")
                time.sleep(5)
                if intento == max_tries - 1:
                    print("Se superó el número máximo de reintentos")
            
geo_dict = {4:"Andalucía", 
            5:"Aragón",
            6:"Cantabria",
            7:"Castilla la Mancha",
            8:"Castilla y León",
            9:"Cataluña",
            10:"País Vasco",
            11:"Principado de Asturias",
            8744:"Comunidad de Ceuta",
            8745:"Comunidad de Melilla",
            13:"Comunidad de Madrid",
            14:"Comunidad de Navarra",
            15:"Comunidad Valenciana",
            16:"Extremadura",
            17:"Galicia",
            8743:"Islas Baleares",
            8742:"Islas Canarias",
            20:"La Rioja",
            21:"Región de Murcia"}


df = pd.DataFrame(registros)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.sort_values(["geo_id","datetime"]).reset_index(drop=True)
df["comunidad"] = df["geo_id"].map(geo_dict)

Pidiendo 2011, geo_id 8744...
Pidiendo 2011, geo_id 8745...
Pidiendo 2011, geo_id 4...
Pidiendo 2011, geo_id 5...
Pidiendo 2011, geo_id 6...
Pidiendo 2011, geo_id 7...
Pidiendo 2011, geo_id 8...
Pidiendo 2011, geo_id 9...
Pidiendo 2011, geo_id 10...
Pidiendo 2011, geo_id 11...
Pidiendo 2011, geo_id 13...
Pidiendo 2011, geo_id 14...
Pidiendo 2011, geo_id 15...
Pidiendo 2011, geo_id 16...
Pidiendo 2011, geo_id 17...
Pidiendo 2011, geo_id 8743...
Pidiendo 2011, geo_id 8742...
Pidiendo 2011, geo_id 20...
Pidiendo 2011, geo_id 21...
Pidiendo 2012, geo_id 8744...
Pidiendo 2012, geo_id 8745...
Pidiendo 2012, geo_id 4...
Pidiendo 2012, geo_id 5...
Pidiendo 2012, geo_id 6...
Pidiendo 2012, geo_id 7...
Pidiendo 2012, geo_id 8...
Pidiendo 2012, geo_id 9...
Pidiendo 2012, geo_id 10...
Pidiendo 2012, geo_id 11...
Pidiendo 2012, geo_id 13...
Pidiendo 2012, geo_id 14...
Pidiendo 2012, geo_id 15...
Pidiendo 2012, geo_id 16...
Pidiendo 2012, geo_id 17...
Pidiendo 2012, geo_id 8743...
Pidiendo 2012, geo

C:\Users\David\AppData\Local\Temp\ipykernel_32332\1545706804.py:69: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["datetime"] = pd.to_datetime(df["datetime"])


In [18]:
df_madrid = df[df["comunidad"] == "Comunidad Valenciana"]
df_madrid

,datetime,value,geo_id,comunidad
1720,2011-01-01 00:00:00+01:00,2526982.724,15,Comunidad Valenciana
1721,2011-02-01 00:00:00+01:00,2271565.119,15,Comunidad Valenciana
1722,2011-03-01 00:00:00+01:00,2373753.059,15,Comunidad Valenciana
1723,2011-04-01 00:00:00+02:00,1961970.722,15,Comunidad Valenciana
1724,2011-05-01 00:00:00+02:00,2095019.601,15,Comunidad Valenciana
...,...,...,...,...
1887,2024-12-01 00:00:00+01:00,2293651.324,15,Comunidad Valenciana
1888,2025-01-01 00:00:00+01:00,2419143.001,15,Comunidad Valenciana
1889,2025-02-01 00:00:00+01:00,2113165.684,15,Comunidad Valenciana
1890,2025-03-01 00:00:00+01:00,2260877.500,15,Comunidad Valenciana
